In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
data=pd.read_csv("c:/Users/Jana Aly/Downloads/first inten project.csv")

In [3]:
data.dtypes

Booking_ID                   object
number of adults              int64
number of children            int64
number of weekend nights      int64
number of week nights         int64
type of meal                 object
car parking space             int64
room type                    object
lead time                     int64
market segment type          object
repeated                      int64
P-C                           int64
P-not-C                       int64
average price               float64
special requests              int64
date of reservation          object
booking status               object
dtype: object

In [4]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

le=LabelEncoder()
data['Booking_ID']=le.fit_transform(data['Booking_ID'])
data['type of meal']=le.fit_transform(data['type of meal'])
data['room type']=le.fit_transform(data['room type'])
data['market segment type']=le.fit_transform(data['market segment type'])
data['date of reservation']=le.fit_transform(data['date of reservation'])
data['booking status']=le.fit_transform(data['booking status'])

In [5]:
data.head()

Booking_ID  number of adults  number of children  number of weekend nights  \
0           0                 1                   1                         2   
1           1                 1                   0                         1   
2           2                 2                   1                         1   
3           3                 1                   0                         0   
4           4                 1                   0                         1   

   number of week nights  type of meal  car parking space  room type  \
0                      5             0                  0          0   
1                      3             3                  0          0   
2                      3             0                  0          0   
3                      2             0                  0          0   
4                      2             3                  0          0   

   lead time  market segment type  repeated  P-C  P-not-C  average price   \
0        224                    3         0    0        0           88.00   
1          5                    4         0    0        0          106.68   
2          1                    4         0    0        0           50.00   
3        211                    4         0    0        0          100.00   
4         48                    4         0    0        0           77.00   

   special requests  date of reservation  booking status  
0                 0                   53               1  
1                 1                  147               1  
2                 0                  236               0  
3                 1                  318               0  
4                 0                  278               0

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
x=data.drop("booking status",axis=1)
y=data["booking status"]

In [8]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=42)

rf_model= RandomForestClassifier(n_estimators=100,max_depth=10,random_state=42)

rf_model.fit(x_train,y_train)

predictions=rf_model.predict(x_test)

accuracy= accuracy_score(y_test,predictions)

print("Accuracy: ",accuracy)

Accuracy:  0.8708832851040375


In [9]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [10]:
param_dist= {
    'n_estimators' :randint(100,500),
    'max_depth':[None,10,20,30],
    'min_samples_split' : randint(2,11)
}

model_tuning= RandomForestClassifier()

random_search_tuning = RandomizedSearchCV(estimator=model_tuning,param_distributions=param_dist,n_iter=50, cv=5, n_jobs=-1, verbose=2)

random_search_tuning.fit(x_train,y_train)

print("Best parameters found: ", random_search_tuning.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 323}


In [11]:
rf_model_tuned=RandomForestClassifier(n_estimators=337,max_depth=20,random_state=42,min_samples_split=2)

rf_model_tuned.fit(x_train,y_train)

predictions_tuned=rf_model_tuned.predict(x_test)

accuracy_tuned= accuracy_score(y_test,predictions_tuned)

print("Accuracy: ",accuracy_tuned)

Accuracy:  0.8978916907813146


In [12]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
x_var = sel.fit_transform(x)

In [14]:
x_var_frame=pd.DataFrame(x_var)
x_var_frame.head()

0    1    2    3    4    5    6      7    8    9       10   11     12
0  0.0  1.0  1.0  2.0  5.0  0.0  0.0  224.0  3.0  0.0   88.00  0.0   53.0
1  1.0  1.0  0.0  1.0  3.0  3.0  0.0    5.0  4.0  0.0  106.68  1.0  147.0
2  2.0  2.0  1.0  1.0  3.0  0.0  0.0    1.0  4.0  0.0   50.00  0.0  236.0
3  3.0  1.0  0.0  0.0  2.0  0.0  0.0  211.0  4.0  0.0  100.00  1.0  318.0
4  4.0  1.0  0.0  1.0  2.0  3.0  0.0   48.0  4.0  0.0   77.00  0.0  278.0

In [18]:
x_trainv,x_testv,y_trainv,y_testv= train_test_split(x_var_frame,y,test_size=0.2,random_state=42,)

rf_model_var= RandomForestClassifier(n_estimators=337,max_depth=20,random_state=42,min_samples_split=2)

rf_model_var.fit(x_trainv,y_trainv)

predictions_var=rf_model_var.predict(x_testv)

accuracy_var= accuracy_score(y_testv,predictions_var)

print("Accuracy: ",accuracy_var)

Accuracy:  0.8961003169353727


In [19]:
param_dist1= {
    'n_estimators' :randint(100,500),
    'max_depth':[None,10,20,30],
    'min_samples_split' : randint(2,11)
}

model_tuning_var= RandomForestClassifier()

random_search_tuning_var = RandomizedSearchCV(estimator=model_tuning_var,param_distributions=param_dist1,n_iter=50, cv=5, n_jobs=-1, verbose=2)

random_search_tuning_var.fit(x_trainv ,y_trainv)

print("Best parameters found: ", random_search_tuning_var.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'max_depth': 30, 'min_samples_split': 3, 'n_estimators': 476}


In [20]:
rf_model_var_tuned= RandomForestClassifier(n_estimators=476,max_depth=30,random_state=42,min_samples_split=3)

rf_model_var_tuned.fit(x_trainv,y_trainv)

predictions_var_tuned=rf_model_var_tuned.predict(x_testv)

accuracy_var_tuned= accuracy_score(y_testv,predictions_var_tuned)

print("Accuracy: ",accuracy_var_tuned)

Accuracy:  0.8961003169353727


Feature selection using correlation analysis

In [21]:
data2=pd.read_csv("c:/Users/Jana Aly/Downloads/first inten project.csv")

In [22]:
data2.head()

Booking_ID  number of adults  number of children  number of weekend nights  \
0   INN00001                 1                   1                         2   
1   INN00002                 1                   0                         1   
2   INN00003                 2                   1                         1   
3   INN00004                 1                   0                         0   
4   INN00005                 1                   0                         1   

   number of week nights  type of meal  car parking space    room type  \
0                      5   Meal Plan 1                  0  Room_Type 1   
1                      3  Not Selected                  0  Room_Type 1   
2                      3   Meal Plan 1                  0  Room_Type 1   
3                      2   Meal Plan 1                  0  Room_Type 1   
4                      2  Not Selected                  0  Room_Type 1   

   lead time market segment type  repeated  P-C  P-not-C  average price   \
0        224             Offline         0    0        0           88.00   
1          5              Online         0    0        0          106.68   
2          1              Online         0    0        0           50.00   
3        211              Online         0    0        0          100.00   
4         48              Online         0    0        0           77.00   

   special requests date of reservation booking status  
0                 0           10/2/2015   Not_Canceled  
1                 1           11/6/2018   Not_Canceled  
2                 0           2/28/2018       Canceled  
3                 1           5/20/2017       Canceled  
4                 0           4/11/2018       Canceled

In [23]:
from scipy.stats import chi2_contingency

In [31]:
numerical_features = data2.select_dtypes(include=['int64', 'float64'])
def correlation_ratio(categories, values):
    categories = np.array(categories)
    values = np.array(values)
    mean_total = values.mean()
    categories_unique = np.unique(categories)
    numerator = 0
    denominator = 0
    for category in categories_unique:
        category_values = values[categories == category]
        mean_category = category_values.mean()
        numerator += len(category_values) * (mean_category - mean_total) ** 2
        denominator += np.sum((category_values - mean_category) ** 2)
    return numerator / denominator if denominator != 0 else 0

correlation_ratios = {}
for column in numerical_features:
    # Drop rows with missing values in either the feature or the target
    valid_rows = data2[['booking status', column]].dropna()
    if valid_rows.shape[0] > 0:
        correlation_ratios[column] = correlation_ratio(valid_rows['booking status'], valid_rows[column])
    else:
        correlation_ratios[column] = 0

correlation_ratios_series = pd.Series(correlation_ratios).sort_values(ascending=False)

# Categorical correlation using Chi-square test
categorical_features = data2.select_dtypes(include=['object', 'category'])
chi2_values = {}

for column in categorical_features:
    if column != 'booking status':
        contingency_table = pd.crosstab(data2[column], data2['booking status'])
        chi2, p, dof, ex = chi2_contingency(contingency_table)
        chi2_values[column] = chi2

chi2_series = pd.Series(chi2_values).sort_values(ascending=False)

# Combine and select top features
combined_results = pd.concat([correlation_ratios_series, chi2_series])
selected_features = combined_results.sort_values(ascending=False).head(3).index

print(selected_features)


Index(['Booking_ID', 'date of reservation', 'market segment type'], dtype='object')


In [32]:
data2_selected=data2.drop(['Booking_ID', 'date of reservation', 'market segment type'],axis=1)

In [33]:
data2_selected.head()

number of adults  number of children  number of weekend nights  \
0                 1                   1                         2   
1                 1                   0                         1   
2                 2                   1                         1   
3                 1                   0                         0   
4                 1                   0                         1   

   number of week nights  type of meal  car parking space    room type  \
0                      5   Meal Plan 1                  0  Room_Type 1   
1                      3  Not Selected                  0  Room_Type 1   
2                      3   Meal Plan 1                  0  Room_Type 1   
3                      2   Meal Plan 1                  0  Room_Type 1   
4                      2  Not Selected                  0  Room_Type 1   

   lead time  repeated  P-C  P-not-C  average price   special requests  \
0        224         0    0        0           88.00                 0   
1          5         0    0        0          106.68                 1   
2          1         0    0        0           50.00                 0   
3        211         0    0        0          100.00                 1   
4         48         0    0        0           77.00                 0   

  booking status  
0   Not_Canceled  
1   Not_Canceled  
2       Canceled  
3       Canceled  
4       Canceled

In [34]:
data22=data2_selected

In [35]:
label=LabelEncoder()

data2_selected['type of meal']=label.fit_transform(data['type of meal'])
data2_selected['room type']=label.fit_transform(data['room type'])
data2_selected['booking status']=label.fit_transform(data['booking status'])

In [36]:
data2_selected.head()

number of adults  number of children  number of weekend nights  \
0                 1                   1                         2   
1                 1                   0                         1   
2                 2                   1                         1   
3                 1                   0                         0   
4                 1                   0                         1   

   number of week nights  type of meal  car parking space  room type  \
0                      5             0                  0          0   
1                      3             3                  0          0   
2                      3             0                  0          0   
3                      2             0                  0          0   
4                      2             3                  0          0   

   lead time  repeated  P-C  P-not-C  average price   special requests  \
0        224         0    0        0           88.00                 0   
1          5         0    0        0          106.68                 1   
2          1         0    0        0           50.00                 0   
3        211         0    0        0          100.00                 1   
4         48         0    0        0           77.00                 0   

   booking status  
0               1  
1               1  
2               0  
3               0  
4               0

In [37]:
x_select=data.drop("booking status",axis=1)
y_select=data["booking status"]

In [47]:
x_trains,x_tests,y_trains,y_tests= train_test_split(x_select,y_select,test_size=0.2,random_state=42)

rf_models= RandomForestClassifier(n_estimators=450,max_depth=30,random_state=42,min_samples_split= 2)

rf_models.fit(x_trains,y_trains)

predictions_select=rf_models.predict(x_tests)

accuracy_select= accuracy_score(y_tests,predictions_select)

print("Accuracy: ",accuracy_select)

Accuracy:  0.8994074686509577


In [41]:
param_dists= {
    'n_estimators' :randint(100,500),
    'max_depth':[None,10,20,30],
    'min_samples_split' : randint(2,11)
}

model_tunings= RandomForestClassifier()

random_search_tunings = RandomizedSearchCV(estimator=model_tunings,param_distributions=param_dists,n_iter=50, cv=5, n_jobs=-1, verbose=2)

random_search_tunings.fit(x_trains,y_trains)

print("Best parameters found: ", random_search_tunings.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'max_depth': 30, 'min_samples_split': 2, 'n_estimators': 308}


Save Model in a file using joblib

In [49]:
print("Before save:",rf_models.predict(x_tests))
import joblib

Before save: [1 1 1 ... 0 1 0]


In [50]:
joblib.dump(rf_models, 'my_model_rf.pkl')

rf_from_joblib = joblib.load('my_model_rf.pkl') 
 
print("After save: ", rf_from_joblib.predict(x_tests))


After save:  [1 1 1 ... 0 1 0]
